<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Ignacio Ponce 
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `http://https://github.com/ignacioponce/mds7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

#Importamos librerias utiles 😸

In [290]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

#Models
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.svm import SVR

from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

# Metricas de evaluación
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import r2_score

#Libreria para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
#!pip install umap-learn

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [33]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [105]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [15]:
#Generamos nueva columna en el DF que actúa como label para clasificación binaria True -> es seleccionado o False -> no es seleccionado
labels=~df_players['National_Position'].isna().rename('Es seleccionado?')
labels.value_counts()

False    16513
True      1075
Name: Es seleccionado?, dtype: int64

Respuesta:
Observamos que se cuenta con registros de 16513 jugadores no seleccionados y 1075 jugadores seleccionados. La muestra es bastante desigual, y podría afectar el clasificador no contar con muestras suficientes para seleccionados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [45]:
#Primero se seleccionan únicamente las columnas de relevancia.
#En este caso, corresponden a todas menos National_Position (labels) y Name  que se considera irrelevante. 
features = df_players.drop(columns = ['Name', 'National_Position'])

#Luego, se definen las transformaciones particulares para cada tipo de columnas

#1) Columnas Categóricas
    #1.1) No ordinales -> Se utiliza un OneHotEncoder, puesto que son variables categóricas no ordinales
categorical_not_ordinal = ['Nationality', 'Club_Position', 'Preffered_Foot']
    #1.2) Ordinales ->Se utiliza un OrdinalEncoder, puesto que son variables categóricas ordinales. En el caso de Weak_foot y Skill_moves no es necesario porque ya están codificadas con enteros del 1 al 5.
categorical_ordinal = ['Work_Rate_Ata', 'Work_Rate_Def']
#Además Work_Rate se debe separar en Attacking y Defending:
features['Work_Rate_Ata']=features['Work_Rate'].str.split(' / ', expand = True)[0]
features['Work_Rate_Def']=features['Work_Rate'].str.split(' / ', expand = True)[1]
features = features.drop(columns = ['Work_Rate'])
category_order = [['Low', 'Medium', 'High'],['Low', 'Medium', 'High']]
        #Work_Rate
        #Weak_foot
        #Skill_Moves

#2) Columnas Numércias
    #2.1) Aspectos físicos de los jugadores -> Se utiliza una estandarización, ya que distribuyen aproximadamente normal, no es necesario manejar outliers.
numerical_normal = ['Age', 'Height', 'Weight']
    #Age
        #Height
        #Weight

    #2.2) Características de juego de los jugadores, en escala 1-100 -> Hay algunas que distribuyen aproximadamente normal, en cuyo caso se utiliza estandarización
         #y hay otras que no distribuyen normal, en cuyo caso se utiliza box-cox aprovechando la positividad de los valores. En ningún caso hay que lidiar con outliers.
        #Estandarización - distribución normal:
numerical_normal = numerical_normal+['Aggression','Reactions','Vision','Composure','Crossing','Short_Pass','Long_Pass',
                                    'Acceleration','Speed','Stamina','Strength','Balance','Agility','Jumping','Volleys']

        #BoxCox - distribución no normal:
numerical_not_normal = ['Ball_Control','Dribbling','Marking','Sliding_Tackle','Standing_Tackle','Interceptions',
                       'Heading','Shot_Power','Finishing','Long_Shots','Curve','Freekick_Accuracy','Penalties']
        

In [82]:
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown= 'ignore'), categorical_not_ordinal),
        ('OrdinalEncoder', OrdinalEncoder(categories = category_order), categorical_ordinal),
        ('StandardScaler', StandardScaler(), numerical_normal),
        ('PowerTransformer', PowerTransformer(method = 'box-cox'), numerical_not_normal)])
                 

In [83]:
#Creamos 3 pipelines:
#Pipeline 1: Linear SVC
pipeline_1 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Model', LinearSVC())])
#Pipeline 2: KNeighbours Classifier
pipeline_2 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Model', KNeighborsClassifier())])
#Pipeline 3: SVC
pipeline_3 = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Model', SVC())])



### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [84]:
#Separación de los datos
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.33, shuffle = True)

In [85]:
#Pipeline 1
pipe_trained_1 = pipeline_1.fit(X_train, y_train)

C:\Users\i5\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [86]:
#Pipeline 2
pipe_trained_2 = pipeline_2.fit(X_train, y_train)

In [87]:
#Pipeline 3
pipe_trained_3 = pipeline_3.fit(X_train, y_train)

In [88]:
pipe_res_1 = pipe_trained_1.predict(X_test)
pipe_res_2 = pipe_trained_2.predict(X_test)
pipe_res_3 = pipe_trained_3.predict(X_test)

In [95]:
print('Pipeline 1')
print(confusion_matrix(y_test, pipe_res_1))
print(classification_report(y_test, pipe_res_1))
print('Pipeline 2')
print(confusion_matrix(y_test, pipe_res_2))
print(classification_report(y_test, pipe_res_2))
print('Pipeline 3')
print(confusion_matrix(y_test, pipe_res_3))
print(classification_report(y_test, pipe_res_3))

Pipeline 1
[[5392   69]
 [ 224  120]]
              precision    recall  f1-score   support

       False       0.96      0.99      0.97      5461
        True       0.63      0.35      0.45       344

    accuracy                           0.95      5805
   macro avg       0.80      0.67      0.71      5805
weighted avg       0.94      0.95      0.94      5805

Pipeline 2
[[5427   34]
 [ 317   27]]
              precision    recall  f1-score   support

       False       0.94      0.99      0.97      5461
        True       0.44      0.08      0.13       344

    accuracy                           0.94      5805
   macro avg       0.69      0.54      0.55      5805
weighted avg       0.92      0.94      0.92      5805

Pipeline 3
[[5461    0]
 [ 342    2]]
              precision    recall  f1-score   support

       False       0.94      1.00      0.97      5461
        True       1.00      0.01      0.01       344

    accuracy                           0.94      5805
   macro avg  

**Respuesta**:

```
Los tres modelos tienen una alta accuracy, sin embargo, al ser tan desigual la distribución de cada clase (muy pocos seleccionados en comparación con no seleccionados), se obtiene una precisión y recall muy alto en la clase False y una precisión y recall más pobres en la clase True. Esto deriva en que el f1-score sea deficiente en esta clase.

Al comparar los tres modelos, se nota que todos tienen un gran desempeño en predecir futbolistas no seleccionados con un f1-score de 0.97. No obstante, al intentar predecir futbolistas seleccionados hay grandes diferencias, siendo el mejor modelo el LineSVC, con un f1-score de 0.45 y el peor modelo el SVC con un f1-score de apenas 0.01

Creo que para mejorar los resultados se deberían contar con más datos de la clase seleccionado, sin embargo, esto parece complejo puesto que la cantidad de futbolistas es finita. Quizá complementar con registros de futbolistas retirados que alguna vez fueron seleccionados, pero esto sería modificar el dominio del origen de la muestra y puede llevar a errores.

También se podría intentar mejorar los resultados probando algún otro modelo o ajustando los hiperparámetros, como la división train/set y/o parámetros particulares del modelo escogido. También se podría evaluar los resultados sobre el X_train, para intentar dilucidar si se tiene Overfitting.
```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [106]:
#Aplicación de etiquetas y cuenta de clases resultantes
groups_position={
    'ST':'ataque','CF':'ataque',
    'RW':'central_ataque','CAM':'central_ataque','LW':'central_ataque',
    'RM':'central','CM':'central','LM':'central',
    'RWB':'central_defensa','CDM':'central_defensa','LWB':'central_defensa',
    'RB':'defensa','CB':'defensa','LB':'defensa',
    'GK':'arquero'}      
df_players['Club_Position']=df_players['Club_Position'].map(groups_position)
df_players_with_position = df_players[df_players['Club_Position'].notna()]
print(df_players_with_position['Club_Position'].value_counts())

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64


In [115]:
#Labels y features
labels = df_players_with_position['Club_Position']
features = df_players_with_position.drop(columns = ['Name', 'National_Position', 'Club_Position'])
features['Work_Rate_Ata']=features['Work_Rate'].str.split(' / ', expand = True)[0]
features['Work_Rate_Def']=features['Work_Rate'].str.split(' / ', expand = True)[1]
features = features.drop(columns = ['Work_Rate'])

In [116]:
#Creación de nuevo pipeline, para esto, utilizaremos el modelo LinearSVC por haber sido el de mejor rendimiento en el item anterior
#Primero reciclamos el ColumnTransformer ya creado para el pre-processing

#La diferencia es que hay que sacar la columna Club_Position:
categorical_not_ordinal = ['Nationality', 'Preffered_Foot']
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown= 'ignore'), categorical_not_ordinal),
        ('OrdinalEncoder', OrdinalEncoder(categories = category_order), categorical_ordinal),
        ('StandardScaler', StandardScaler(), numerical_normal),
        ('PowerTransformer', PowerTransformer(method = 'box-cox'), numerical_not_normal)])

pipeline_multiclase = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Model', LinearSVC())])


In [117]:
#Separación de train set y test set
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.33, shuffle = True)

In [118]:
#Entrenamiento
pipe_trained_multiclase = pipeline_multiclase.fit(X_train, y_train)

C:\Users\i5\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [119]:
#Predicción
pipe_res_multiclase = pipe_trained_multiclase.predict(X_test)

In [120]:
#Evaluación
print('Pipeline Multiclase')
print(confusion_matrix(y_test, pipe_res_multiclase))
print(classification_report(y_test, pipe_res_multiclase))

Pipeline Multiclase
[[201   0   0   0   0   0]
 [  0 117  15   6   0   2]
 [  0  18 166  66  11  42]
 [  0  15  88  69   6   7]
 [  0   0   8   7  14  41]
 [  0   0  16   3   8 374]]
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       201
         ataque       0.78      0.84      0.81       140
        central       0.57      0.55      0.56       303
 central_ataque       0.46      0.37      0.41       185
central_defensa       0.36      0.20      0.26        70
        defensa       0.80      0.93      0.86       401

       accuracy                           0.72      1300
      macro avg       0.66      0.65      0.65      1300
   weighted avg       0.70      0.72      0.71      1300



**Respuesta**:

```
Los resultados obtenidos para cada clase dependen fuertemente de la cantidad de muestras que se tiene de estas. En particular, se nota lo siguiente:

- En el caso de los arqueros, estos tienen características muy particulares que hace que se logren identificar con total precisión f1-score perfecto.
- Los mejores resultados se obtienen en predicción de defensas (según f1-score), justamente la clase sobre la que se tiene mayor cantidad de registros: 1180.
- Los peores resultados se obtienen en predicción de central_defensa (según f1-score), justamente la clase sobre la que se tiene menor cantidad de registros: 209.

- Los resultados son aceptables para ciertas clases y para otras no. Para intentar mejorar el desempeño se podría evaluar la utilización de otro modelo, modificar hiperparámetros, verificar que no esté haciendo overfitting, conseguir más datos, entre otras acciones
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

Siguiendo la guía para escoger un modelo, se selecciona Ridge Regression preliminarmente, puesto que a priori no es clara la decisión de si sólo unas pocas variables importan en el sueldo de un jugador




In [213]:
#Lectura de datos y join con tabla original
sueldos = pd.read_csv('salarios.csv', index_col = 1).drop(columns = ['Unnamed: 0'])
df_players_with_salary = df_players.join(sueldos, on = 'Name')
#Sólo nos quedamos con jugadores de los cuales se conocía el dato de su salario
df_players_with_salary = df_players_with_salary[df_players_with_salary['Weekly Salary'].notna()]


In [214]:
#Labels y features
features = df_players_with_salary.drop(columns = ['Name', 'Weekly Salary'])
features['Work_Rate_Ata']=features['Work_Rate'].str.split(' / ', expand = True)[0]
features['Work_Rate_Def']=features['Work_Rate'].str.split(' / ', expand = True)[1]
labels = df_players_with_salary[['Weekly Salary']]

In [215]:
#Utilizamos un MinMaxScaler sobre Weekly Salary
minmax_scaler = MinMaxScaler()
labels = minmax_scaler.fit_transform(labels)

In [216]:
#Reciclamos el Column Transformer anterior para el preprocessing, pero añadiendo la columnas faltantes
categorical_not_ordinal = ['Nationality', 'Club_Position', 'Preffered_Foot', 'National_Position']

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown= 'ignore'), categorical_not_ordinal),
        ('OrdinalEncoder', OrdinalEncoder(categories = category_order), categorical_ordinal),
        ('StandardScaler', StandardScaler(), numerical_normal),
        ('PowerTransformer', PowerTransformer(method = 'box-cox'), numerical_not_normal)])

In [291]:
#Creamos pipeline

pipeline_regresion = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('Model', RidgeCV())])#Luego de algunas pruebas, se determinó que tuvo un mejor desempeño RidgeCV en lugar de Ridge


In [312]:
#Separamos train set y test set
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.33, shuffle = True)

In [313]:
#Entrenamos pipeline
pipe_trained_regresion = pipeline_regresion.fit(X_train, y_train)

In [314]:
#Predecimos con test set
pipe_res_regresion = pipe_trained_regresion.predict(X_test)

In [315]:
#Evaluamos de los resultados
print('Pipeline Regresión, puntaje R2:')
#print(confusion_matrix(y_test, pipe_res_regresion))
print(r2_score(y_test, pipe_res_regresion))

Pipeline Regresión, puntaje R2:
0.37863794560393205


**Respuesta**:

```
    $R^2$ consiste en una métrica para la evaluación de la calidad de un regresor. Un alto coeficiente $R^2$ indica que los datos se ajustan muy al modelo, mientras que un bajo $R^2$ indica lo contrario.
   
   Luego de algunas iteraciones, se determinó que tiene un mejor desempeño RidgeCV por sobre Ridge, consiguiendo un puntaje superior a 0.35, cumpliendo así con el requerimiento.
   
   Para mejorar los resultados pienso que se requiriría una mezcla de ambas cosas, más datos y más características. Las que se tienen están muy enfocadas en una caracterización completa sobre las capacidades futbolísticas de los jugadores, y algunos datos básicos como nacionalidad, edad, etc... Creo que es muy relevante otro aspecto para la determinación de sus salarios, como:
   - La liga en la que juegan (sueldos muy distintos según el nivel competitivo de la liga)
   - El equipo en el que juegan (a veces hay equipos en ligas poco completitivas que pagan altísimos sueldos (caso arabia saudita, MLS, china, entre otras)
   - La "marca" del jugador, que se podría expresar por ejemplo, a través de alguna estadística de su publicidad, venta de camisetas, etc... (hay equipos que contratan jugadores y les dan un alto sueldo no sólo por sus capacidades futbolísticas sino por el arrastre a la afición, venta de camisetas y cuestiones de marketing que estos generan).
   - Entre otros...
   
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>